# Router Benchmark on PYNQ-Z2

This notebook demonstrates how to run the router benchmark and read cycle counts from the FPGA.

**LED Mapping:**
- LED0 = Base12 (condition 2)
- LED1 = Router (condition 3) — should light with default latencies
- LED2 = Base10 (condition 1)
- LED3 = Base2 (condition 0)

**Prerequisites:**
- Copy `router_bench.bit` and `router_bench.hwh` to the same directory as this notebook
- Or update the `overlay_path` below to point to the correct location

In [ ]:
from pynq import Overlay, MMIO
import time

# Load the overlay
overlay_path = "router_bench.bit"
ol = Overlay(overlay_path)

print("Overlay loaded successfully!")

In [ ]:
# Create MMIO object for the benchmark module
BASE_ADDR = 0x43C00000
mmio = MMIO(BASE_ADDR, 0x1000)

# Register offsets
CTRL   = 0x00  # Control: bit0=start, bit1=soft_clear
STAT   = 0x04  # Status: bit0=running, bit1=done, bits[3:2]=winner_code
T0     = 0x08  # Cycle count for condition 0 (Base2)
T1     = 0x0C  # Cycle count for condition 1 (Base10)
T2     = 0x10  # Cycle count for condition 2 (Base12)
T3     = 0x14  # Cycle count for condition 3 (Router)
ONEHOT = 0x18  # Internal one-hot winner (cond0..3)

print("MMIO initialized at address 0x{:08X}".format(BASE_ADDR))

In [ ]:
def start_benchmark():
    """Start the benchmark by writing bit0=1 to CTRL register"""
    mmio.write(CTRL, 0x1)

def read_status():
    """Read status register and decode fields"""
    v = mmio.read(STAT)
    running     = (v & 0x1) != 0
    done        = (v & 0x2) != 0
    winner_code = (v >> 2) & 0x3
    return running, done, winner_code

def wait_done(timeout_s=2.0):
    """Wait for benchmark to complete with timeout"""
    t0 = time.time()
    while True:
        running, done, winner_code = read_status()
        if done:
            return winner_code
        if (time.time() - t0) > timeout_s:
            raise TimeoutError("Benchmark did not finish")
        time.sleep(0.001)

def read_times():
    """Read cycle counts for all 4 conditions"""
    return (mmio.read(T0), mmio.read(T1), mmio.read(T2), mmio.read(T3))

print("Helper functions defined")

In [ ]:
# Condition labels with LED mapping
cond_labels = {
    0: "Base2   (LED3)",
    1: "Base10  (LED2)",
    2: "Base12  (LED0)",
    3: "Router  (LED1)",
}

print("Running benchmark...")
start_benchmark()
winner = wait_done()
t = read_times()

print("\n" + "="*50)
print("Benchmark Results")
print("="*50)
print("\nCycle totals:")
print(f"  Base2   (cond0): {t[0]:6d} cycles")
print(f"  Base10  (cond1): {t[1]:6d} cycles")
print(f"  Base12  (cond2): {t[2]:6d} cycles")
print(f"  Router  (cond3): {t[3]:6d} cycles")
print(f"\nWinner: cond{winner} → {cond_labels[winner]}")
print("="*50)

In [ ]:
# Optional: Run multiple times and average
import numpy as np

num_runs = 10
results = []

print(f"Running benchmark {num_runs} times...")
for i in range(num_runs):
    start_benchmark()
    winner = wait_done()
    t = read_times()
    results.append(t)
    print(f"Run {i+1}: Base2={t[0]}, Base10={t[1]}, Base12={t[2]}, Router={t[3]} → Winner: cond{winner}")

# Calculate averages
avg = np.mean(results, axis=0)
std = np.std(results, axis=0)

print("\n" + "="*50)
print(f"Average over {num_runs} runs:")
print("="*50)
print(f"  Base2   (cond0): {avg[0]:6.1f} ± {std[0]:4.1f} cycles")
print(f"  Base10  (cond1): {avg[1]:6.1f} ± {std[1]:4.1f} cycles")
print(f"  Base12  (cond2): {avg[2]:6.1f} ± {std[2]:4.1f} cycles")
print(f"  Router  (cond3): {avg[3]:6.1f} ± {std[3]:4.1f} cycles")
print("="*50)

In [ ]:
# Optional: Visualize results
import matplotlib.pyplot as plt

conditions = ['Base2\n(cond0)', 'Base10\n(cond1)', 'Base12\n(cond2)', 'Router\n(cond3)']
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']

plt.figure(figsize=(10, 6))
bars = plt.bar(conditions, avg, yerr=std, color=colors, alpha=0.7, capsize=5)
plt.ylabel('Cycles', fontsize=12)
plt.title('Router Benchmark Results (Average over {} runs)'.format(num_runs), fontsize=14)
plt.grid(axis='y', alpha=0.3)

# Highlight the winner
min_idx = np.argmin(avg)
bars[min_idx].set_edgecolor('red')
bars[min_idx].set_linewidth(3)

plt.tight_layout()
plt.show()

print(f"Winner: {conditions[min_idx].replace(chr(10), ' ')} with {avg[min_idx]:.1f} cycles")